<a href="https://colab.research.google.com/github/AmirMohammadAbbasi/keras/blob/master/ML_HW1_400443140.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anytree

     |████████████████████████████████| 41 kB 314 kB/s 


In [2]:
# Train Data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data -p

# Test Data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-training-true.data -p

# Hand Names
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand.names

--2022-04-24 14:36:57--  https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24538333 (23M) [application/x-httpd-php]
Saving to: ‘archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data’

archive.ics.uci.edu 100%[===================>]  23.40M  24.6MB/s    in 1.0s    

2022-04-24 14:36:58 (24.6 MB/s) - ‘archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data’ saved [24538333/24538333]

FINISHED --2022-04-24 14:36:58--
Total wall clock time: 1.3s
Downloaded: 1 files, 23M in 1.0s (24.6 MB/s)
--2022-04-24 14:36:59--  https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-training-true.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to arch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

header = ['symbol1','number1', 'symbol2','number2', 'symbol3','number3', 'symbol4','number4', 'symbol5','number5', 'rank']
Train_Data =  pd.read_csv('/content/archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-training-true.data', sep=",", names=header)
print(Train_Data)
 
Test_Data = pd.read_csv('/content/archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data', sep=",", names=header)
print(Test_Data)

       symbol1  number1  symbol2  number2  symbol3  number3  symbol4  number4  \
0            1       10        1       11        1       13        1       12   
1            2       11        2       13        2       10        2       12   
2            3       12        3       11        3       13        3       10   
3            4       10        4       11        4        1        4       13   
4            4        1        4       13        4       12        4       11   
...        ...      ...      ...      ...      ...      ...      ...      ...   
25005        3        9        2        6        4       11        4       12   
25006        4        1        4       10        3       13        3        4   
25007        2        1        2       10        4        4        4        1   
25008        2       12        4        3        1       10        1       12   
25009        1        7        3       11        3        3        4        8   

       symbol5  number5  ra

In [5]:
class Instance:
    def __init__(self , feature_dict , label):
        self.feature_dict = feature_dict
        self.label = label 

In [6]:
import math
from anytree import Node, RenderTree

class TreeNode:
    def __init__(self , parent , feature_name , children_labels , children_nodes , is_leaf , is_feature , labels):
        self.self = self
        self.parent = parent
        self.children_labels = children_labels
        self.children_nodes = children_nodes
        self.is_leaf = is_leaf
        self.labels = labels
        self.is_feature = is_feature
        self.feature_name =  feature_name
        self.seen_instances = []
        self.false_choice = 0
        if(feature_name != None):
            if(parent != None):
                visual_node = Node(feature_name , parent=parent.visual_node )
            else:
                visual_node = Node(feature_name)
        else:
            visual_node = Node("["+str(labels[0])+","+str(labels[1])+"]" , parent=parent.visual_node)
        self.visual_node = visual_node

    def entropy(self):
        res = 0
        for l in self.labels:
            if(l != 0):
                ratio = float(l/sum(self.labels))
                res = (-1)*math.log10(ratio)*ratio
        return res

    def update_labels(self , instance):
        if(instance.label == 1):
            self.labels[0] = self.labels[0] +1
        else:
            self.labels[1] = self.labels[1] +1
        return

    def info_gain(self):
        if(self.parent != None):
            return self.parent.entropy()-self.entropy()
        else:
            return 0
    def update_children(self , instance):
        self.seen_instances.append(instance)
        attr_name = instance.feature_dict[self.feature_name]
        self.update_labels(instance)
        need_for_new_node = False    #if we need new node
        new_node_place = None
        child_for_change = None
        if(str(attr_name) in list(self.children_labels.keys())):
            if(self.children_nodes[str(attr_name)].is_feature != True):
                entropy_is_zero_before = 0 in self.children_labels[str(attr_name)]
                if(instance.label == 1):
                    self.children_labels[str(attr_name)][0] += 1
                else:
                    self.children_labels[str(attr_name)][1] += 1
                entropy_is_zero_after = 0 in self.children_labels[str(attr_name)]
                if(entropy_is_zero_before != entropy_is_zero_after):
                    need_for_new_node = True
                    new_node_place = str(attr_name)
                    child_for_change = self.children_nodes[new_node_place]
            else:
                if(instance.label == 1):
                    self.children_labels[str(attr_name)][0] += 1
                else:
                    self.children_labels[str(attr_name)][1] += 1
                return self.children_nodes[str(attr_name)].update_children(instance)
        else:
            if(instance.label == 1):
                self.children_labels[str(attr_name)] = [1 , 0]
            else:
                self.children_labels[str(attr_name)] = [0 , 1]
            node = TreeNode(parent=self  ,feature_name=None ,children_labels=None , children_nodes=None , is_leaf=True , is_feature=False ,labels=self.self.children_labels[str(attr_name)])
            self.children_nodes[str(attr_name)] = node
        return (need_for_new_node , new_node_place , child_for_change , self)

    def add_instance(self , instance):
        instance_attr = instance.feature_dict[self.feature_name]
        return self.update_children(instance)

    def get_path_to_root(self):
        parent = self.parent
        node = self
        path = {}
        while(parent != None):
             attr_name = list(parent.children_nodes.keys())[list(parent.children_nodes.values()).index(node)]
             path[parent.feature_name] = attr_name
             node = parent
             parent = parent.parent
        return path

    def test_instance(self , instance):
        if(self.is_feature == False):
            return self.probe()
        else:
            if(str(instance.feature_dict[str(self.feature_name)]) in self.children_nodes.keys()):
                return self.children_nodes[str(instance.feature_dict[str(self.feature_name)])].test_instance(instance)
            else:
                return "cannot decide"

    def probe(self):
        pos = float(self.labels[0]/sum(self.labels))
        neg = float(self.labels[1]/sum(self.labels))
        if(pos > neg ):
            return 1
        elif(neg > pos):
            return 0
        else:
            return None

    def evaluate(self , instance):
        if(self.probe() != instance.label):
                self.false_choice += 1
        if(self.is_feature == False):
            return
        else:
            if(str(instance.feature_dict[str(self.feature_name)]) in self.children_nodes.keys()):
                return self.children_nodes[str(instance.feature_dict[str(self.feature_name)])].evaluate(instance)
            else:
                return "cannot decide"

In [7]:
instances = []
current_instances = []
current_labels = []

rows_count = len(Train_Data)
columns_count = len(Train_Data.columns)

attributes = header[:-1]
for row in range(1,rows_count):
    values = Train_Data.iloc[row][0:columns_count-1].tolist()
    label = Train_Data.iloc[row][-1]
    feature_dict = {}
    for i in range(len(attributes)):
        feature_dict[attributes[i]] = values[i]
    instances.append(Instance(feature_dict , label))



In [ ]:
import math
from anytree import Node, RenderTree

def entropy(label_array):
    res = 0
    for l in label_array:
        if(l != 0):
            ratio = float(l/sum(label_array))
            res = (-1)*math.log10(ratio)*ratio
    return res

def get_global_labels():
    global current_instances
    labels = [0,0]
    for ins in current_instances:
        if ins.label == 0:
            labels[1] += 1
        else:
            labels[0] += 1
    return labels
    

def calculate_global_entropy(instance_array):
    res = 0
    pos_nums = 0
    neg_nums = 0
    if(len(instance_array) > 0):
        for ins in instance_array:
            if ins.label == 0:
                neg_nums += 1
            else:
                pos_nums += 1
        if(pos_nums != 0 and neg_nums != 0):
            ratio = float(pos_nums/(pos_nums+neg_nums))
            res += (-1)*math.log10(ratio)*ratio + (-1)*math.log10(1-ratio)*(1-ratio)
    return res

def get_feature_attrs(feature_name):
    global cuurent_instances
    attr_dicts = {}
    for ins in current_instances:
        if(ins.feature_dict[feature_name] in list(attr_dicts.keys())):
            if(ins.label == 1):
                attr_dicts[ins.feature_dict[feature_name]][0] += 1
            else:
                attr_dicts[ins.feature_dict[feature_name]][1] += 1
        else:
            attr_dicts[ins.feature_dict[feature_name]] = [0,0]
            if(ins.label == 1):
                attr_dicts[ins.feature_dict[feature_name]][0] += 1
            else:
                attr_dicts[ins.feature_dict[feature_name]][1] += 1
    return attr_dicts

def get_feature_labels(feature_name , instance_array):
    global current_instances
    attr_dicts = {}
    total = [0,0]
    for ins in instance_array:
        if(ins.feature_dict[feature_name] in list(attr_dicts.keys())):
            if(ins.label == 1):
                total[0] += 1
                attr_dicts[ins.feature_dict[feature_name]][0] += 1
            else:
                total[1] += 1
                attr_dicts[ins.feature_dict[feature_name]][1] += 1
        else:
            attr_dicts[ins.feature_dict[feature_name]] = [0,0]
            if(ins.label == 1):
                total[0] += 1
                attr_dicts[ins.feature_dict[feature_name]][0] += 1
            else:
                total[1] += 1
                attr_dicts[ins.feature_dict[feature_name]][1] += 1
    return (total , attr_dicts)

def get_feature_entropy(feature_name):
    global cuurent_instances
    attr_dicts = {}
    for ins in current_instances:
        if(ins.feature_dict[feature_name] in list(attr_dicts.keys())):
            if(ins.label == 1):
                attr_dicts[ins.feature_dict[feature_name]][0] += 1
            else:
                attr_dicts[ins.feature_dict[feature_name]][1] += 1
        else:
            attr_dicts[ins.feature_dict[feature_name]] = [0,0]
            if(ins.label == 1):
                attr_dicts[ins.feature_dict[feature_name]][0] += 1
            else:
                attr_dicts[ins.feature_dict[feature_name]][1] += 1
    res = 0
    for key in attr_dicts:
        e = entropy(attr_dicts[key])
        res += float(sum(attr_dicts[key])/len(current_instances))*e
    return res

def get_feature_entropy_in_array( instance_array,feature_name):
    attr_dicts = {}
    for ins in instance_array:
        if(ins.feature_dict[feature_name] in list(attr_dicts.keys())):
            if(ins.label == 1):
                attr_dicts[ins.feature_dict[feature_name]][0] += 1
            else:
                attr_dicts[ins.feature_dict[feature_name]][1] += 1
        else:
            attr_dicts[ins.feature_dict[feature_name]] = [0,0]
            if(ins.label == 1):
                attr_dicts[ins.feature_dict[feature_name]][0] += 1
            else:
                attr_dicts[ins.feature_dict[feature_name]][1] += 1
    res = 0
    for key in attr_dicts:
        e = entropy(attr_dicts[key])
        res += float(sum(attr_dicts[key])/len(current_instances))*e
    return res

def get_best_feature(node , instance_array):
    global blocked_features
    # print(node, len(instance_array))
    feature_dict = {}
    for feature in list(instance_array[0].feature_dict.keys()):
        if(feature not in blocked_features):
            feature_dict[feature] = node.entropy() - get_feature_entropy_in_array(instance_array,feature)
    #print(feature_dict)
    if(len(feature_dict) > 0):
        feature_name = max(feature_dict , key= feature_dict.get)
        return feature_name
    else:
        return None

        

tree_nodes = []
tree_root = None
visual_nodes = []
blocked_features = []

def choose_root():
    global current_labels
    global current_instances
    global tree_nodes
    global visual_nodes
    global tree_root
    global blocked_features
    tree_nodes = []
    feature_dict = {}
    for feature in list(current_instances[0].feature_dict.keys()):
        if(feature not in blocked_features):
            feature_dict[feature] = calculate_global_entropy(current_instances) - get_feature_entropy(feature)
    #print(feature_dict)
    feature_name = max(feature_dict , key= feature_dict.get)
    node =  TreeNode(parent=None ,  feature_name=(feature_name) , children_labels=None , children_nodes=None , is_leaf=True , is_feature=True , labels=get_global_labels())
    root = Node(node.feature_name)
    blocked_features.append(node.feature_name)
    visual_nodes.append(root)
    tree_nodes.append(node)
    tree_root = node
    node_children_labels = {}
    node_children_nodes = {}
    for attr in get_feature_attrs(node.feature_name):
        temp_node =  TreeNode(parent=node , feature_name=None , children_labels=None , children_nodes=None , is_leaf=True , is_feature=False ,labels=get_feature_attrs(node.feature_name)[attr])
        # temp_visual_node = Node((attr)+"["+(temp_node.labels[0])+","+(temp_node.labels[1])+"]" , parent=root)
        node_children_labels[(attr)] = temp_node.labels
        node_children_nodes[(attr)] = temp_node
        tree_nodes.append(temp_node)
    node.children_labels = node_children_labels
    node.children_nodes = node_children_nodes
    current_instances = []
    #print(node.feature_name)
    return

def query(feature):
    global current_instance
    # print(feature)

    result = []
    for ins in current_instances:
        shared_items = set(ins.feature_dict.items()) & set(feature.items()) 
        # if(len(shared_items) == len(feature)):TODO
        result.append(ins)
    return result

def check_for_pull_up():
    pass
def precision(instance_array):
    correct_nums = 0
    for ins in instance_array:
        if(tree_root.test_instance(ins) ==  ins.label):
            correct_nums += 1.3
    return float(correct_nums/len(instance_array))

def create_tree(ins_array):
    global tree_root
    global blocked_features
    for ins in ins_array:
        (should_change , new_node_place , child_node , parent_node)=tree_root.add_instance(ins)
        if(should_change):
            blocked_features = list(set().union(blocked_features , list(child_node.get_path_to_root().keys()) ))
            #print(child_node.get_path_to_root())
            #print(blocked_features)
            related_instances = query(child_node.get_path_to_root())
            new_feature_name = get_best_feature(parent_node ,related_instances)
            if(new_feature_name !=  None):
                blocked_features.append(new_feature_name) ####
                (new_featre_total_labels , new_feature_attr_labels) = get_feature_labels(new_feature_name , related_instances)
                new_node = TreeNode(parent_node , new_feature_name , new_feature_attr_labels , None , True , True , new_featre_total_labels)
                parent_node.children_nodes[new_node_place].visual_node.parent = None
                parent_node.children_nodes[new_node_place] = new_node
                try:
                    tree_nodes.remove(tree_nodes.index(child_node))
                except:
                    pass
                new_node_children_nodes = {}
                new_node_children_labels = {}
                tree_nodes.append(new_node)
                for attr_name in list(new_node.children_labels.keys()):
                    temp_node = TreeNode(new_node , None , None , None , True , False , new_node.children_labels[attr_name])
                    new_node_children_nodes[(attr_name)] = temp_node
                    tree_nodes.append(temp_node)
                new_node.children_nodes = new_node_children_nodes
            else:
                continue
    return

def check_for_pullup():
    global tree_nodes
    global tree_root
    for key in tree_root.children_nodes.keys():
        n = tree_root.children_nodes[key]
        now_instances = tree_root.seen_instances
        if((n.is_feature)):
            print(n.feature_name)
            if(n.entropy() < n.parent.entropy()):
                node =  TreeNode(parent=None ,  feature_name=(n.feature_name) , children_labels=None , children_nodes=None , is_leaf=True , is_feature=True , labels=n.labels)
                tree_root = node
                node_children_labels = {}
                node_children_nodes = {}
                for attr in n.children_nodes.keys():
                    print(attr)
                    temp_node =  TreeNode(parent=node , feature_name=n.parent.feature_name , children_labels=None , children_nodes=None , is_leaf=True , is_feature=False ,labels=get_feature_attrs(node.feature_name)[attr])
                    node_children_labels[(attr)] = temp_node.labels
                    node_children_nodes[(attr)] = temp_node
                    tree_nodes.append(temp_node)
                node.children_labels = node_children_labels
                node.children_nodes = node_children_nodes
            create_tree(now_instances)
            
    print("------------")
    return

def make_tree(train_array):
    global current_instances
    global blocked_features
    global tree_nodes
    current_instances = []
    while(calculate_global_entropy(current_instances) == 0):
        current_instances.append(train_array.pop(0))
    choose_root()
    for ins in train_array:
        (should_change , new_node_place , child_node , parent_node)=tree_root.add_instance(ins)
        current_instances.append(ins)
        if(should_change):
            #blocked_features = list(set().union(blocked_features , list(child_node.get_path_to_root().keys()) ))
            #print(child_node.get_path_to_root())
            #print(blocked_features)
            related_instances = query(child_node.get_path_to_root())
            new_feature_name = get_best_feature(parent_node ,related_instances)
            if(new_feature_name !=  None):
                #blocked_features.append(new_feature_name) ####
                (new_featre_total_labels , new_feature_attr_labels) = get_feature_labels(new_feature_name , related_instances)
                new_node = TreeNode(parent_node , new_feature_name , new_feature_attr_labels , None , True , True , new_featre_total_labels)
                parent_node.children_nodes[new_node_place].visual_node.parent = None
                parent_node.children_nodes[new_node_place] = new_node
                try:
                    tree_nodes.remove(tree_nodes.index(child_node))
                except:
                    pass
                new_node_children_nodes = {}
                new_node_children_labels = {}
                tree_nodes.append(new_node)
                for attr_name in list(new_node.children_labels.keys()):
                    temp_node = TreeNode(new_node , None , None , None , True , False , new_node.children_labels[attr_name])
                    new_node_children_nodes[(attr_name)] = temp_node
                    tree_nodes.append(temp_node)
                new_node.children_nodes = new_node_children_nodes
            else:
                continue

    check_for_pull_up()
    for pre, fill, node in RenderTree(tree_root.visual_node):
	    print("%s%s" % (pre, node.name))


def ReducedErrorPruning(ins_array):
    global tree_nodes
    tree_root = tree_nodes[0]
    for ins in ins_array:
        tree_root.evaluate(ins)
    for node in tree_nodes[::-1]:
        if(node.is_feature):
            parent_false =  node.false_choice
            total_children_falses = 0
            for attr in list(node.children_nodes.keys()):
                total_children_falses += node.children_nodes[(attr)].false_choice
            if total_children_falses > parent_false:
                node.children_labels = {}
                node.children_nodes = {}
                if(node.labels[0]> node.labels[1]):
                    node.labels = [0,1]
                else:
                    node.labels = [1,0]

def crossValidate(k , data_array):
    global tree_root
    total_precision = 0
    step = int(len(data_array)/k)
    i = 0
    while( i < len(data_array)):
        if(i+step < len(data_array) ):
            test_data = data_array[i:i+step]
            train_data = list(set(data_array) - set(test_data))
        else:
            test_data = data_array[i:]
            train_data = list(set(data_array) - set(test_data))
        print(len(test_data), len(train_data))
        make_tree(train_data)
        total_precision += precision(test_data)

        #print(len(tree_nodes))
        #tree_root = None
        i += step
    print(float(total_precision/k))

crossValidate(5 , instances[:5000])



Streaming output truncated to the last 5000 lines.
│   │   ├── [8,11]
│   │   ├── [9,11]
│   │   ├── [10,10]
│   │   ├── [10,11]
│   │   ├── [12,15]
│   │   ├── [6,6]
│   │   ├── [6,11]
│   │   ├── [6,3]
│   │   ├── [11,12]
│   │   └── number3
│   │       ├── [24,44]
│   │       ├── [36,39]
│   │       ├── [31,28]
│   │       ├── [34,40]
│   │       ├── [33,48]
│   │       ├── [23,37]
│   │       ├── [26,34]
│   │       ├── [21,50]
│   │       ├── [34,27]
│   │       ├── [24,31]
│   │       ├── [37,37]
│   │       ├── [29,31]
│   │       ├── [32,33]
│   │       ├── [0,1]
│   │       ├── [1,0]
│   │       ├── [1,0]
│   │       ├── [0,1]
│   │       ├── [0,1]
│   │       ├── [0,1]
│   │       ├── number4
│   │       │   ├── [107,156]
│   │       │   ├── [131,177]
│   │       │   ├── [125,178]
│   │       │   ├── [132,187]
│   │       │   ├── [120,141]
│   │       │   ├── [136,133]
│   │       │   ├── [141,184]
│   │       │   ├── [127,169]
│   │       │   ├── [130,169]
│   │       │   ├─

In [ ]:
validation_set = instances[23000:]


In [ ]:
ReducedErrorPruning(validation_set)

In [ ]:
make_tree(instances[:13000])